# Using the Bailo module 

To connect to the API, you will need to authenticate the Bailo client.  

If you have your .env configured (see README), instantiating Bailo is simple:

```
    bailo = Bailo()
```

Otherwise, for Cognito authentication:
```
    bailo = Bailo(
        bailo_url=bailo_url,
        cognito_client_id=cognito_client_id,
        cognito_client_secret=cognito_client_secret,
        cognito_username=cognito_username,
        cognito_pwd=cognito_pwd,
        cognito_region=cognito_region,
        cognito_user_pool_id=cognito_user_pool_id,
    )
```

For PKI:
```
    bailo = Bailo(
        bailo_url=bailo_url, 
        pki_ca=pki_ca, 
        pki_p12=pki_p12
    )
```


If you don't need a client and just want to make use of model loading/bundling functionality, set client=False

```
    bailo = Bailo(client=False)
```


### Instantiate Bailo
This example sets up the client connection via a .env file

In [ ]:
from bailoclient import Bailo

bailo = Bailo()

### Upload a model

You will need to provide:
* Model metadata - see the minimal metadata for information on what fields this should include
* Model binary - the actual model
* Model code - additional files required alongside your model (model.py, requirements.txt etc)
* AWS gateway (optional) - set aws_gateway=False if you haven't already configured a .env file and you are not uploading via an AWS gateway

#### View minimum required metadata:

In [ ]:
bailo.minimal_metadata

#### Upload the model

In [ ]:
from pkg_resources import resource_filename

uploaded_model = bailo.upload_model(
    metadata=bailo.minimal_metadata,
    binary_file=resource_filename("bailoclient", "resources/minimal_code.zip"),
    code_file=resource_filename("bailoclient", "resources/minimal_code.zip"),
)

print(f"Created new model: {uploaded_model}")

# Get the model card for the uploaded model
uploaded_model_card = bailo.get_model_card(model_uuid=uploaded_model['uuid'])

### View models

#### Get all models and output their ids

In [ ]:
models = bailo.get_models()
model_uuids = [m.uuid for m in models]

print(f"Available models: {model_uuids}")

#### View a model card

Model cards can be pretty-printed with the display() method.

In [ ]:
model_card = bailo.get_model_card(model_uuid=uploaded_model['uuid'])

model_card.display()

You can also output the model card to text

In [ ]:
readable_json = model_card.display(to_screen=False)

### Deployments

* View deployments
* Request a deployment
* Download mode code and binary for a deployment


In [ ]:
deployment_metadata = bailo.minimal_deployment_metadata
deployment_metadata['highLevelDetails']['modelID'] = uploaded_model['uuid']

deployment = bailo.request_deployment(deployment_metadata)

print(f"Requested deployment: {deployment}")

In [ ]:
user_deployments = bailo.get_my_deployments()

print(f"User deployments: {user_deployments}")

In [ ]:
uploaded_model_latest_version = uploaded_model_card['latestVersion']['metadata']['highLevelDetails']['modelCardVersion']

if user_deployments:
    deployment = user_deployments[0]

    resp = bailo.download_model_files(
        deployment_uuid=deployment["uuid"],
        model_version=uploaded_model_latest_version,
        file_type="code",
    )

# Example workflow

Create a very basic sklearn model trained on the Iris dataset and which returns a numeric label for predictions.

In [ ]:
from sklearn import datasets
from sklearn.svm import SVC

iris = datasets.load_iris()
clf = SVC()
clf.fit(iris.data, iris.target)

Bundle the model files via the Bailo model bundling functionality for sklearn models. 

This will save the sklearn model as a pkl file and put all the required files into the code and binary zips for uploading to Bailo. 

In [ ]:
from bailoclient import Bailo
bailo = Bailo()

output_path = "./sklearn_example"

bailo.bundle_model(model=clf, output_path=output_path, model_flavour="sklearn")

Upload the sklearn model to Bailo via the client. 

In [ ]:
model_binary = f"{output_path}/binary.zip" # or replace ith minimal binary
model_code = f"{output_path}/code.zip" # or replace with minimal code

# set some of the metadata for the model
model_metadata = bailo.minimal_metadata
model_metadata['highLevelDetails']['name'] = "sklearn model"
model_metadata['highLevelDetails']['modelInASentence'] = "predicts iris data"
model_metadata['highLevelDetails']['modelOverview'] = "sklearn model to predict iris data"

# upload the model
uploaded_model = bailo.upload_model(
    metadata=model_metadata,
    binary_file=model_binary,
    code_file=model_code,
)

Check that the model has been uploaded

In [ ]:
# user_models = bailo.get_my_models()
user_models = bailo.get_models()

for model in user_models:
    print(model.uuid)

Say we want to update the model with a new version that returns the actual string label rather than the numeric value when we call predict.

Because we don't need to change any of the code files, instead of using Bailo bundling functionality, we'll just pickle this model ourselves and zip it

In [ ]:
import pickle
import zipfile
import os


## Update the model to return labels
clf.fit(iris.data, iris.target_names[iris.target])

## Save the new model binary and zip
os.makedirs(f"{output_path}/new_binary", exist_ok=True)

with open(f"{output_path}/new_binary/model.pkl", "wb") as f:
    pickle.dump(clf, f)

zipfile.ZipFile(f"{output_path}/new_binary/binary.zip", mode="w").write(f"{output_path}/new_binary/model.pkl")

We then need to update the model metadata. We can get the metadata from the model card we retrieved when we called bailo.get_my_models()

To view the fields on the model card:
```
    dir(model_card)
```

To look at the validation schema for the model:
```
    schema = bailo.get_model_schema(model_uuid)
```

In [ ]:
## Get model and required fields for updating (UUID and metadata)
model_card = user_models[0]
model_uuid = model_card.uuid
metadata = model_card.latestVersion.metadata


## Update some of the metadata fields for the new model version
metadata.highLevelDetails.name = f"Updated sklearn model"
metadata.highLevelDetails.modelCardVersion = "v2.0"

## Validate the model card
result = model_card.validate()
for error in result.errors:
    print(f"{error.field}: {error.description}")

Now we can push our new model version up to Bailo

In [ ]:
update_resp = bailo.update_model(
    metadata=metadata,
    model_uuid=model_uuid,
    binary_file=f"{output_path}/new_binary/binary.zip",
    code_file=f"{output_path}/code.zip", # code is unchanged
)

Let's have a look at the models now and check that it's been updated

In [ ]:
user_models = bailo.get_models()
latest_model = user_models[0]

latest_model.latestVersion.metadata.highLevelDetails.name

We can now request a deployment of our model - to do this we have to provide metadata relating to our deployment request.

We can access the minimal deployment metadata on the Bailo interface.

In [ ]:
deployment_metadata = bailo.minimal_deployment_metadata
deployment_metadata

Let's edit this metadata to request a deployment for our model

In [ ]:
import datetime

# name the deployment
deployment_metadata['highLevelDetails']['name'] = "sklearn test deployment"

# set end date to tomorrow
end_date = str(datetime.date.today() + datetime.timedelta(days=1))
deployment_metadata['highLevelDetails']['endDate']['date'] = end_date

# set model ID to our new model's uuid
deployment_metadata['highLevelDetails']['modelID'] = model_uuid

# set owner to current user id
deployment_metadata['contacts']['owner'][0]['id'] = bailo.get_me().id

deployment_metadata


In [ ]:
deployment_request = bailo.request_deployment(deployment_metadata)

With an **approved** deployment you can request to download the model files. 

To carry out this step you will need your deployment request to be approved. This is not currently possible via the Bailo Python module.

By default the download_model_files function will download the model binary, but you can also download the model code files.

In [ ]:
deployment_uuid=deployment_request['uuid']
model_version = latest_model.latestVersion.metadata.highLevelDetails.modelCardVersion

bailo.download_model_files(deployment_uuid=deployment_uuid, model_version=model_version, output_dir=f"{output_path}/downloaded_binary")

Now we can unzip the model files, load in the model and run some predictions

In [ ]:
loaded_model = bailo.load_model(f"{output_path}/downloaded_binary/sklearn_example/new_binary/model.pkl", model_flavour="sklearn")

In [ ]:
loaded_model.predict(iris.data)